In [1]:
import torch

import sys
import yaml
import torchvision
from torchvision import transforms, datasets

import numpy as np
import os
from sklearn import preprocessing
from torch import cuda
from torch.utils.data.dataloader import DataLoader

In [2]:
from tqdm import tqdm

In [3]:
sys.path.append('../')
from mymodels.resnet_base_network import ResNet18
from mydata.imageloader import MyDataset, psnrDataUnit

In [4]:
batch_size = 32
data_transforms = torchvision.transforms.Compose([transforms.ToTensor()])

In [5]:
config = yaml.load(open("../config/config.yaml", "r"), Loader=yaml.FullLoader)

In [6]:
import random

pair_list = []
file_path = "/home/hong/dir1/PyTorch-BYOL/writing2.txt"

f = open(file_path, 'r')
lines = f.readlines()
for line in lines:
    listed = line.split(" ")
    tmp_unit = psnrDataUnit(listed[0], listed[1], listed[2], listed[3])
    pair_list.append(tmp_unit)
f.close()


In [7]:
image_path = '/mnt/URP_DS/HR' # without last slash
test_data_path = '/mnt/URP_DS/HR'  #without last slash

img1_idx = 0


# create the test_image_paths
test_imgs_psnr_list = []
for p_unit in pair_list:
    test_imgs_psnr_list.append((test_data_path+'/'+p_unit.getimg1()+'.png', test_data_path+'/'+p_unit.getimg2()+'.png', p_unit.getonetwo()))

# shuffle the pairs for training
random.shuffle(pair_list)

train_imgs_psnr_list = []
for p_unit in pair_list:
    train_imgs_psnr_list.append((image_path+'/'+p_unit.getimg1()+'.png', image_path+'/'+p_unit.getimg2()+'.png', p_unit.getonetwo()))
# split train valid from train paths (80,20), (1, 99) -> just for convention
train_imgs_psnr_list, valid_img_psnr_list = train_imgs_psnr_list[:int(0.5*len(train_imgs_psnr_list))], train_imgs_psnr_list[int(0.5*len(train_imgs_psnr_list)):]
test_imgs_psnr_list, valid2_img_psnr_list = test_imgs_psnr_list[:int(0.5*len(test_imgs_psnr_list))], test_imgs_psnr_list[int(0.5*len(test_imgs_psnr_list)):]

print('train_image_path example: ', train_imgs_psnr_list[img1_idx])
print('test_image_path example: ', test_imgs_psnr_list[img1_idx])
print(len(train_imgs_psnr_list), len(test_imgs_psnr_list))

train_image_path example:  ('/mnt/URP_DS/HR/93_1.png', '/mnt/URP_DS/HR/106_16.png', 36.082514090000004)
test_image_path example:  ('/mnt/URP_DS/HR/124_18.png', '/mnt/URP_DS/HR/68_21.png', 44.17431062)
50731 50731


In [8]:
# device = 'cpu'
device = 'cuda' if cuda.is_available() else 'cpu'
print(f"Training with: {device}")
if device=='cuda':
    torch.cuda.empty_cache()

train_dataset = MyDataset(train_imgs_psnr_list, transform=data_transforms)
test_dataset = MyDataset(test_imgs_psnr_list, transform=data_transforms)

Training with: cuda


In [9]:
print("Input shape:", train_dataset[0][0].shape)

Input shape: torch.Size([3, 512, 512])


In [10]:
train_loader = DataLoader(train_dataset, batch_size=batch_size,
                          num_workers=0, drop_last=False, shuffle=True, pin_memory=True)

test_loader = DataLoader(test_dataset, batch_size=batch_size,
                          num_workers=0, drop_last=False, shuffle=False, pin_memory=True)

In [11]:
encoder = ResNet18(**config['network'])
output_feature_dim = encoder.projetion.net[0].in_features
print(output_feature_dim)

512


In [12]:
#load pre-trained parameters
load_params = torch.load(os.path.join('/home/hong/dir1/PyTorch-BYOL/runs/Sep26_15-10-29_mango2/checkpoints/model.pth'),
                        map_location=torch.device(torch.device(device)))

if 'online_network_state_dict' in load_params:
    encoder.load_state_dict(load_params['online_network_state_dict'])
    print("Parameters successfully loaded.")

# remove the projection head
encoder = torch.nn.Sequential(*list(encoder.children())[:-1])    
encoder = encoder.to(device)

Parameters successfully loaded.


In [21]:
# Neural Network Class
class MyOne(torch.nn.Module):
    def __init__(self, D_in, H1, H2, H3, H4, H5, D_out):
        super().__init__()
        self.linear1 = torch.nn.Linear(D_in, H1)
        self.linear2 = torch.nn.Linear(H1, H2)
        self.linear3 = torch.nn.Linear(H2, H3)
        self.linear4 = torch.nn.Linear(H3, H4)
        self.linear5 = torch.nn.Linear(H4, H5)
        self.linear6 = torch.nn.Linear(H5, D_out)
    
    def forward(self, x):
        x = torch.nn.functional.relu(self.linear1(x))
        x = torch.nn.functional.relu(self.linear2(x))
        x = torch.nn.functional.relu(self.linear3(x))
        x = torch.nn.functional.relu(self.linear4(x))
        x = torch.nn.functional.relu(self.linear5(x))
        x = self.linear6(x)
        return x    

In [22]:
# logreg = LogisticRegression(output_feature_dim*2, 10)
# logreg = logreg.to(device)
mymo = MyOne(output_feature_dim*2, output_feature_dim*2, output_feature_dim*2,output_feature_dim*2, output_feature_dim*2, output_feature_dim, 1)
mymo = mymo.to(device)
# 모델의 state_dict 출력
print("Model's state_dict:")
for param_tensor in mymo.state_dict():
    print(param_tensor, "\t", mymo.state_dict()[param_tensor].size())

Model's state_dict:
linear1.weight 	 torch.Size([1024, 1024])
linear1.bias 	 torch.Size([1024])
linear2.weight 	 torch.Size([1024, 1024])
linear2.bias 	 torch.Size([1024])
linear3.weight 	 torch.Size([1024, 1024])
linear3.bias 	 torch.Size([1024])
linear4.weight 	 torch.Size([1024, 1024])
linear4.bias 	 torch.Size([1024])
linear5.weight 	 torch.Size([512, 1024])
linear5.bias 	 torch.Size([512])
linear6.weight 	 torch.Size([1, 512])
linear6.bias 	 torch.Size([1])


In [15]:
def get_features_from_encoder(encoder, loader):
    
    x_train = []
    y_train = []

    # get the features from the pre-trained model
    # for i, (x1, x2, y) in enumerate(tqdm(loader)):
    
    for i, (x1, x2, y) in enumerate(tqdm(loader)):
        x1=x1.to(device)
        x2=x2.to(device)
        # y=y.to(device)
        with torch.no_grad():
            f_vector1 = encoder(x1)
            f_vector2 = encoder(x2)
            feature_vector = torch.cat((f_vector1, f_vector2), 1)
            feature_vector=feature_vector.to('cpu')
            # y_train=y_train.to('cpu')

            x_train.extend(feature_vector)
            y_train.extend(y)

        # print(x1.shape, x2.shape, feature_vector.shape, y.shape)

            
    x_train = torch.stack(x_train)
    y_train = torch.tensor(y_train)
    return x_train, y_train

In [16]:
encoder.eval()
x_train, y_train = get_features_from_encoder(encoder, train_loader)
x_test, y_test = get_features_from_encoder(encoder, test_loader)

if len(x_train.shape) > 2:
    print(x_train.shape)
    x_train = torch.mean(x_train, dim=[2, 3])
    x_test = torch.mean(x_test, dim=[2, 3])
    
print("Training data shape:", x_train.shape, y_train.shape)
print("Testing data shape:", x_test.shape, y_test.shape)

100%|██████████| 1586/1586 [24:40<00:00,  1.07it/s]


torch.Size([50731, 1024, 10, 10])
Training data shape: torch.Size([50731, 1024]) torch.Size([50731])
Testing data shape: torch.Size([50731, 1024]) torch.Size([50731])


In [17]:
def create_data_loaders_from_arrays(X_train, y_train, X_test, y_test):

    train = torch.utils.data.TensorDataset(X_train, y_train)
    train_loader = torch.utils.data.DataLoader(train, batch_size=64, shuffle=True)

    test = torch.utils.data.TensorDataset(X_test, y_test)
    test_loader = torch.utils.data.DataLoader(test, batch_size=1, shuffle=False)
    return train_loader, test_loader

In [18]:
scaler = preprocessing.StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train).astype(np.float32)
x_test = scaler.transform(x_test).astype(np.float32)

In [19]:
train_loader, test_loader = create_data_loaders_from_arrays(torch.from_numpy(x_train), y_train, torch.from_numpy(x_test), y_test)

In [23]:
running_loss_history = []
running_logit_history = []

optimizer = torch.optim.Adam(mymo.parameters(), lr=3e-4)
# criterion = torch.nn.CrossEntropyLoss()
criterion = torch.nn.L1Loss()
eval_every_n_epochs = 10

# device = 'cuda' if cuda.is_available() else 'cpu'
# print(f"Training with: {device}")
for epoch in tqdm(range(200)):
#     train_acc = []
    for x, y in train_loader:

        x = x.to(device)
        y = y.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad() 
        
        logits = mymo(x)
        running_logit_history.append(1/logits[0].item())
        # predictions = torch.argmax(logits, dim=1)
        
        loss = criterion(logits, y.unsqueeze(1))
        running_loss_history.append(loss)
        
        loss.backward()
        optimizer.step()

    if epoch % eval_every_n_epochs == 0:
        print(loss.item())
################################################################


  0%|          | 1/200 [00:02<09:19,  2.81s/it]

0.0013875875156372786


  6%|▌         | 11/200 [00:32<09:43,  3.09s/it]

0.0010060096392408013


 10%|█         | 21/200 [01:02<08:46,  2.94s/it]

...

 90%|█████████ | 181/200 [08:49<00:53,  2.83s/it]

0.00024686832330189645


 96%|█████████▌| 191/200 [09:17<00:23,  2.66s/it]

0.0002486768935341388


100%|██████████| 200/200 [09:44<00:00,  2.92s/it]


In [24]:
test_result_diff = []
for x, y in tqdm(test_loader):
    x = x.to(device)
    y = y.to(device)
    logits = mymo(x)
    ty = 1/float(y[0].item())
    oy = 1/logits[0].item()
    test_result_diff.append((oy, ty, ty-oy))

with open('half2_5layer_1029'+'.txt', 'w') as f:
    for item1, item2, item3 in test_result_diff:
        f.write("%s, %s, diff: %s\n" % (item1, item2, item3))
    
    

100%|██████████| 50731/50731 [00:24<00:00, 2099.52it/s]
